# Setup environment
This section installs all of the required dependencies and imports libraries

In [20]:
# install required libraries and dependencies
!pip install fhir.resources
!pip install openai
import json
from datetime import datetime
from openai import OpenAI
from google.colab import userdata
from fhir.resources.patient import Patient
from fhir.resources.observation import Observation
from fhir.resources.codeableconcept import CodeableConcept
from fhir.resources.coding import Coding
from fhir.resources.bundle import Bundle, BundleEntry
from fhir.resources.questionnaireresponse import QuestionnaireResponse
from fhir.resources.questionnaire import Questionnaire
from fhir.resources.humanname import HumanName

The following script will extract observation resources from a questionnaire response resource.

# Services
This section contains all of the functions that extract information from the FHIR Questionnaire (Q) and QuestionnaireResponses(QRs) into Patient and Observation resources.  

## Questionnaire Extraction
This section contains all of the functions for extracting information.

In [25]:
def extract_units(questionnaire_file):
    units_map = {}
    with open(questionnaire_file, 'r') as f:
        questionnaire = json.load(f)
        for item in questionnaire['item']:
            link_id = item['linkId']
            if 'extension' in item:
                for extension in item['extension']:
                    if 'url' in extension and extension['url'] == 'http://hl7.org/fhir/StructureDefinition/questionnaire-unit':
                        units_map[link_id] = extension['valueCoding']['display']
    return units_map

def extract_patient_id(patient_dict):
    if 'id' in patient_dict:
        return patient_dict['id']
    else:
        raise ValueError("Patient Id not found")

def extract_answers_from_qr(questionnaire, response):
    # Create a mapping from linkId to code from the Questionnaire
    linkid_to_code = {item.linkId: next((code.code for code in item.code if code.code), None) for item in questionnaire.item}

    extracted_values = []

    # Iterate through each response item
    for item in response.item:
        linkId = item.linkId
        code = linkid_to_code.get(linkId, 'Unknown Code')  # Get the code associated with the linkId, default to 'Unknown Code'
        if item.answer:  # Ensure there is at least one answer
            answer = item.answer[0]

            # Directly extracting the value based on its type
            value = None
            if hasattr(answer, 'valueString') and answer.valueString is not None:
                value = answer.valueString
            elif hasattr(answer, 'valueCoding') and hasattr(answer.valueCoding, 'display') and answer.valueCoding.display is not None:
                value = answer.valueCoding.display
            elif hasattr(answer, 'valueDate') and answer.valueDate is not None:
                value = answer.valueDate.isoformat()  # Convert date to string format if it's a datetime object

            # Append the result as a tuple of (linkId, code, value)
            if value is not None:
                extracted_values.append((linkId, code, value))

    return extracted_values

def json_to_fhir(questionnaire_file, response_file):
    """Converts JSON files into FHIR resources."""
    with open(questionnaire_file, 'r') as file:
        questionnaire_data = json.load(file)
    questionnaire = Questionnaire.parse_obj(questionnaire_data)

    with open(response_file, 'r') as file:
        response_data = json.load(file)
    response = QuestionnaireResponse.parse_obj(response_data)

    return questionnaire, response

## Instantiate FHIR Resources
This section contains the functions that will instantiate the various FHIR resources using the fhir.resources library.

In [46]:
def create_observation(link_id, text, answer, units=None, patient_info=None):
    # Set code representing the question
    code = CodeableConcept(
        coding=[Coding(
            system="http://loinc.org",
            code=link_id.lstrip('/'),
            display=text
        )]
    )

    # Set status to 'final' as these are completed observations
    observation = Observation(code=code, status="final")

    # Set patient information
    if patient_info:
        observation.subject = {
            "reference": patient_info['reference'],
            "display": patient_info['display']
        }
    else:
      print(patient_info)

    # Set value based on the answer type
    if 'valueDecimal' in answer:
        value = float(answer['valueDecimal'])
        if units:  # Check if units are provided
            observation.valueQuantity = {
                "value": value,
                "unit": units
            }
        else:
            observation.valueQuantity = {"value": value}
    elif 'valueString' in answer:
        observation.valueString = answer['valueString']
    elif 'valueCodeableConcept' in answer:
        value_codeable_concept = CodeableConcept(
            coding=[Coding(
                system=answer['valueCodeableConcept']['system'],
                code=answer['valueCodeableConcept']['code'],
                display=answer['valueCodeableConcept']['display']
            )]
        )
        observation.valueCodeableConcept = value_codeable_concept

    return observation

def instantiate_patient_from_answers(extracted_answers):
    """Instantiates and returns a new Patient resource using extracted answers with dictionary mappings."""
    patient = Patient()

    # seed specific data elements for name and id
    patient.name = [HumanName(use='official')]
    patient.id = '8247202'

    # Mapping dictionary to convert codes to patient attributes
    attribute_mappings = {
        'first-name': ('name[0].given', lambda x: [x]),
        'lastname': ('name[0].family', lambda x: x),
        'sexatbirth': ('gender', lambda x: x.lower()),
        'dob': ('birthDate', lambda x: x)  # Assuming the date is already in the correct format
    }

    # Apply mappings to set patient attributes
    for linkId, code, value in extracted_answers:
        if code in attribute_mappings:
            attribute_path, transform = attribute_mappings[code]
            # Using exec to set the attribute dynamically
            exec(f"patient.{attribute_path} = transform(value)")

    return patient

def consolidate_resources(patient, observations_list):
    # instantiate a bundle of type transaction
    bundle = Bundle(type = "transaction")

    # Create a bundle entry for the patient
    patient_entry = BundleEntry(
        resource=patient,
        request={
            "method": "POST",
            "url": "Patient"
        }
    )
    bundle.entry = [patient_entry]

    # Create bundle entries for each observation
    for observation in observations_list:
        observation_entry = BundleEntry(
            resource=observation,
            request={
                "method": "POST",
                "url": "Observation"
            }
        )
        bundle.entry.append(observation_entry)

    # Print the bundle
    print("\n**Here is the consolidated FHIR bundle**")
    print(bundle.json(indent=4))

    return bundle

## LLM-based SOAP Note Generation

In [50]:
def call_llm(bundle):
    #Initialize OpenAI client
    client = OpenAI(api_key=userdata.get('openai'))

    # add a prompt to the bundle
    prompt = f"Convert to SOAP note style with proper medical terminology, creating a section for each letter but do not include assessment and plan nor any inference, which contains FHIR resources, into a short summary note for a physician: {bundle}. The bundle is a FHIR resource, please omit FHIR language and reference to LOINC codes. It should always be structured as Subjective: text then Objective: text. For the subjective section, the start should be (Patient Name) is a (age)(gender) who presents for... For objective, list only the finding values."

    try:
        # Make a completion request to GPT-3
        response = client.completions.create(
            model="gpt-3.5-turbo-instruct",
            prompt=prompt,
            max_tokens=500
        )

        # Get the generated text from the response
        print(response.choices[0].text)
        # return response.choices[0].text

    except Exception as e:
        # Handle exceptions and print an error message
        print(f"Error: {e}")
        return None

## Helper functions
This section contains various helper functions to compartmentalize the code into more digestable chunks.

In [44]:
def process_patient(pat_q, pat_qr):
    # convert the json files to a fhir object in the fhir.resources library
    questionnaire, response = json_to_fhir(pat_q, pat_qr)

    # extract answers with codes from the QR
    extracted_values = extract_answers_from_qr(questionnaire, response)

    # instantiate the patient resource using extracted values
    patient = instantiate_patient_from_answers(extracted_values)

    # Print the extracted patient resource
    print("\n**Here is our extracted patient resource**")
    print(patient.json(indent=4))

    return patient

def process_observations(obs_q, obs_qr):
    # Extract units from questionnaire file
    units_map = extract_units(obs_q)

    # Parse questionnaire response JSON file
    with open(obs_qr, 'r') as f:
        questionnaire_response = json.load(f)

    # Extract patient information
    patient_info = {
        "reference": questionnaire_response['subject']['reference'],
        "display": questionnaire_response['subject']['display']
    }

    # instantiate list of observations
    obs_list = []

    # Iterate through each item in the questionnaire response
    print("\n**Here is our extracted observation resources**")
    for item in questionnaire_response['item']:
        link_id = item['linkId']
        text = item['text']
        answer = item.get('answer', [{}])[0]

        # Get units for the current question
        units = units_map.get(link_id)

        # Create Observation resource for the item
        observation = create_observation(link_id, text, answer, units, patient_info)

        # add created observation to list
        obs_list.append(observation)

        # Print the observation
        print(observation.json(indent=4))

    return obs_list

# Main
This section contains the main code that calls upon other functions.

In [53]:
def main():
    # Load all of the questionnaire (Q) and questionnaireResponses (QR)
    pat_q = 'patient-q.json'
    pat_qr = 'patient-qr.json'
    obs_q = 'ht_wt_panel-q.json'
    obs_qr = 'ht_wt_panel-qr.json'

    # Process patient Q and QR
    patient = process_patient(pat_q, pat_qr)

    # Process the observation Q and QR
    observations_list = process_observations(obs_q, obs_qr)

    # Consolidate resources into a single bundle
    bundle = consolidate_resources(patient, observations_list)

    # Generate the list of references that were used
    response = call_llm(bundle)

if __name__ == "__main__":
    main()


**Here is our extracted patient resource**
{
    "resourceType": "Patient",
    "id": "8247202",
    "name": [
        {
            "use": "official",
            "family": "Simpson",
            "given": [
                "Bart"
            ]
        }
    ],
    "gender": "male",
    "birthDate": "2000-06-05"
}

**Here is our extracted observation resources**
{
    "resourceType": "Observation",
    "status": "final",
    "code": {
        "coding": [
            {
                "system": "http://loinc.org",
                "code": "10154-3",
                "display": "What brings you to the doctor today?"
            }
        ]
    },
    "subject": {
        "reference": "Patient/8247202",
        "display": "Bart Simpson"
    },
    "valueString": "I have pain in my stomach"
}
{
    "resourceType": "Observation",
    "status": "final",
    "code": {
        "coding": [
            {
                "system": "http://loinc.org",
                "code": "29463-7",
            